<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src_v4/CE__CNN_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time
import json
import joblib
import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score

tf.random.set_seed(42)

tf.__version__

'2.4.1'

In [2]:
CONFIG = {
    'timestamp'             : str(datetime.datetime.now()),
    'model'                 : 'Deep Learning CNN',
    'test_fold'             : 'Fold 1',
    'frame_length'          : 400,
    'frame_step'            : 200,
    'fft_length'            : 400,
    'architecture'          : '',
    'batch_size'            : 32,
    'epochs'                : 0,
    'learning_rate'         : 0.00001,
    'patience'              : 3,
    'training_time'         : 0,
    'testing_time'          : 0,
    'cm_atick'              : '',
    'cr_atick'              : '',
    'cm_valentina'          : '',
    'cr_valentina'          : ''
}

In [3]:
BASE_DIR_AF         = '/content/drive/MyDrive/Research/Crowd Emotion v4/'
BASE_DIR_VAL        = '/content/drive/MyDrive/Research/Crowd Emotion Val/'
LOG_FILE            = '/content/drive/MyDrive/Research/Crowd Emotion Logs/dl_cnn.txt'

FOLDS               = ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5']
# FOLDS               = ['']
EMOTIONS            = ['Approval', 'Disapproval', 'Neutral']

# DATASET_SHAPE       = (14882, 219, 201)
DATASET_SHAPE       = (9488, 219, 201)

WINDOWS_DIR         = 'Windows_AF/'
DATASET_DIR         = 'Dataset_STFT/'

STFT_CONFIG         = 'FL_400_FS_200_FFT_400'

In [4]:
# %%time

# count = 0

# X = np.zeros(DATASET_SHAPE, dtype='float16')
# y = np.zeros((DATASET_SHAPE[0], 1), dtype='uint8')
# f = np.zeros((DATASET_SHAPE[0], ), dtype='uint8') 
# c = np.zeros((DATASET_SHAPE[0], ), dtype='uint8') 

# for emotion in EMOTIONS:
#     print('processing audio for ' + emotion, end=' ... ')

#     for fold in FOLDS:
#         fold_path = os.path.join(
#             BASE_DIR_VAL,
#             WINDOWS_DIR,
#             emotion,
#             fold
#         )
#         filenames = os.listdir(fold_path)

#         for filename in filenames:
#             file_path = os.path.join(fold_path, filename)
#             data = joblib.load(file_path)

#             for i in range(data.shape[0]):
#                 x = data[i, :]
#                 spectrogram = tf.signal.stft(
#                    signals        = x,
#                    frame_length   = CONFIG['frame_length'],
#                    frame_step     = CONFIG['frame_step'],
#                    fft_length     = CONFIG['fft_length'],
#                    window_fn      = tf.signal.hamming_window
#                 )
                
#                 magnitude = tf.abs(spectrogram).numpy()

#                 X[count, :] = magnitude
#                 y[count, :] = EMOTIONS.index(emotion)
#                 # f[count] = FOLDS.index(fold)
#                 # c[count] = int(filename[7:11])

#                 count += 1

#     print('√')

In [5]:
# joblib.dump(X, os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_X.joblib'))
# joblib.dump(y, os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_y.joblib'))
# joblib.dump(f, os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_f.joblib'))
# joblib.dump(c, os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_c.joblib'))

In [6]:
# joblib.dump(X, os.path.join(BASE_DIR_VAL, DATASET_DIR, STFT_CONFIG + '_X.joblib'))
# joblib.dump(y, os.path.join(BASE_DIR_VAL, DATASET_DIR, STFT_CONFIG + '_y.joblib'))

In [7]:
X = joblib.load(os.path.join(BASE_DIR_AF, DATASET_DIR, STFT_CONFIG + '_X.joblib'))
y = joblib.load(os.path.join(BASE_DIR_AF, DATASET_DIR, STFT_CONFIG + '_y.joblib'))
f = joblib.load(os.path.join(BASE_DIR_AF, DATASET_DIR, STFT_CONFIG + '_f.joblib'))
c = joblib.load(os.path.join(BASE_DIR_AF, DATASET_DIR, STFT_CONFIG + '_c.joblib'))

In [8]:
mask      = (f == FOLDS.index(CONFIG['test_fold']))
train_X   = X[~mask, :]
train_y   = y[~mask, :]
val_X     = X[mask, :]
val_y     = y[mask, :]

In [9]:
del X

In [10]:
test_X = joblib.load(os.path.join(BASE_DIR_VAL, DATASET_DIR, STFT_CONFIG + '_X.joblib'))
test_y = joblib.load(os.path.join(BASE_DIR_VAL, DATASET_DIR, STFT_CONFIG + '_y.joblib'))

In [11]:
train_X, train_y = shuffle(train_X, train_y)

In [12]:
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(train_X)

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(219, 201, 1)),
    tf.keras.layers.experimental.preprocessing.Resizing(32, 32), 
    norm_layer,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3),
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 32, 32, 1)         0         
_________________________________________________________________
normalization (Normalization (None, 32, 32, 201)       403       
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        57920     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0

In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=CONFIG['learning_rate']),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
%%time
start_time = time.time()

history = model.fit(
    train_X,
    train_y, 
    validation_data = (val_X, val_y),  
    epochs          = 300,
    callbacks       = tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=CONFIG['patience'],
        restore_best_weights=True
    )
)

training_time = time.time() - start_time

Epoch 1/300
375/375 [==============================] - 15s 13ms/step - loss: 46.5968 - accuracy: 0.4287 - val_loss: 2.6826 - val_accuracy: 0.6542
Epoch 2/300
375/375 [==============================] - 3s 9ms/step - loss: 5.7274 - accuracy: 0.5268 - val_loss: 1.0042 - val_accuracy: 0.7197
Epoch 3/300
375/375 [==============================] - 3s 9ms/step - loss: 2.0403 - accuracy: 0.5490 - val_loss: 0.8114 - val_accuracy: 0.7714
Epoch 4/300
375/375 [==============================] - 3s 9ms/step - loss: 1.2873 - accuracy: 0.5592 - val_loss: 0.7470 - val_accuracy: 0.7777
Epoch 5/300
375/375 [==============================] - 3s 9ms/step - loss: 1.0097 - accuracy: 0.5773 - val_loss: 0.6975 - val_accuracy: 0.7881
Epoch 6/300
375/375 [==============================] - 3s 9ms/step - loss: 0.9401 - accuracy: 0.5947 - val_loss: 0.7026 - val_accuracy: 0.7856
Epoch 7/300
375/375 [==============================] - 3s 9ms/step - loss: 0.8911 - accuracy: 0.6067 - val_loss: 0.6843 - val_accuracy: 0.7

In [ ]:
%%time

start_time = time.time()

loss, accuracy = model.evaluate(test_X, test_y)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

testing_time = time.time() - start_time

In [ ]:
metrics = history.history
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
# ------------------ Testing on Aticks's data -----------------

y_true = val_y
y_pred = np.argmax(model.predict(val_X), axis=1)

result_atick = classification_report(y_true, y_pred)
print(result_atick)

In [ ]:
confusion_mtx_atick = tf.math.confusion_matrix(y_true, y_pred) 
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx_atick, xticklabels=EMOTIONS, yticklabels=EMOTIONS, 
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

In [ ]:
# ------------------ Testing on Valentina's data -----------------

y_true = test_y
y_pred = np.argmax(model.predict(test_X), axis=1)

result_valentina = classification_report(y_true, y_pred)
print(result_valentina)

In [ ]:
confusion_mtx_valentina = tf.math.confusion_matrix(y_true, y_pred) 
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx_valentina, xticklabels=EMOTIONS, yticklabels=EMOTIONS, 
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

In [ ]:
summary = []
model.summary(print_fn=lambda x: summary.append(x))
CONFIG['architecture'] = summary
CONFIG['epochs'] = max(history.epoch)
CONFIG['training_time'] = training_time
CONFIG['testing_time'] = testing_time
CONFIG['cm_atick'] = np.array2string(confusion_mtx_atick.numpy())
result_list_atick = result_atick.split('\n')
CONFIG['cr_atick'] = result_list_atick
CONFIG['cm_valentina'] = np.array2string(confusion_mtx_valentina.numpy())
result_list_valentina = result_valentina.split('\n')
CONFIG['cr_valentina'] = result_list_valentina

In [ ]:
config = json.dumps(CONFIG, indent=4)
print(config)

In [ ]:
f = open(LOG_FILE, 'a')
f.write('\n')
f.write(config)
f.write('\n')
f.close()